In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

## load two raw submission file to look for common errors

In [2]:
df = pd.read_csv("../data/raw.csv")
ef = pd.read_csv("../data/raw_new.csv")

In [3]:
df = df.sort_values("ID").fillna("").reset_index(drop=True)
ef = ef.sort_values("ID").fillna("").reset_index(drop=True)

In [4]:
df["gen_len"] = df["CodeList"].apply(lambda x: len(x.split()))
ef["gen_len"] = ef["CodeList"].apply(lambda x: len(x.split()))

In [5]:
t1 = mlb.fit_transform(df["CodeList"])

In [6]:
mlb.classes_

array([' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',',
       '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F',
       'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
       'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a',
       'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
       'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{',
       '|', '}'], dtype=object)

In [7]:
t2 = mlb.fit_transform(ef["CodeList"])

In [8]:
mlb.classes_

array([' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',',
       '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F',
       'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
       'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a',
       'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
       'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{',
       '|', '}'], dtype=object)

In [9]:
df["val"] = (t1-t2).sum(axis=1)
ef["val"] = (t1-t2).sum(axis=1)

In [10]:
print(df.iloc[52]["Text"])

In recent years, there's been a trend towards more efficient and streamlined development processes. This has been driven by the need for faster delivery times and higher-quality software. One way this is being achieved is through the use of Continuous Integration/Continuous Deployment (CI/CD) pipelines. These are automated processes that build, test, and deploy software whenever changes are made. For example, a basic pipeline configuration in YAML might look like pipeline, stages, build, test, deploy. This approach helps to catch issues early and reduce the time it takes to get software into the hands of users.


## wrong rows with short generation length

In [11]:
ids = df[(df["val"]!=0) & (df["gen_len"] < 2)]["ID"]
df[(df["val"]!=0) & (df["gen_len"] < 2)]


,ID,Text,ContainsCode,CodeList,gen_len,val
152,153,"In software development, it's not just about w...",<class 'str'>,,0,-9
411,412,Software development has changed significantly...,<class 'str'>,JavaScript,1,-10
729,730,The importance of commenting in code cannot be...,<class 'str'>,//,1,1
737,738,"In software engineering, a design pattern is a...",<class 'str'>,getInstance(),1,10
795,796,Software development methodologies have evolve...,<class 'str'>,,0,-19
1475,1476,Software development is a fascinating field. I...,<class 'str'>,,0,-17
1638,1639,Software development is a dynamic field that i...,<class 'str'>,C++,1,-28
1645,1646,Software development is a complex and multifac...,<class 'str'>,TypeScript,1,-21
1869,1870,Software development is a fascinating field. F...,<class 'str'>,,0,-21
1873,1874,Software development has a multitude of strate...,<class 'str'>,,0,-7


In [150]:
df[df["ID"].isin(ids)].to_dict(orient="records")

[{'ID': 153,
  'Text': "In software development, it's not just about what you can do, but also how you do it. Good coding practices, like writing clear, readable code, using version control, and testing your code regularly, can make a big difference in the long run. \n For example, in JavaScript, it's considered good practice to use 'use strict' at the beginning of your scripts. This directive helps catch common coding mistakes and 'unsafe' actions such as using undeclared variables.",
  'ContainsCode': "<class 'str'>",
  'CodeList': '',
  'gen_len': 0,
  'val': -9},
 {'ID': 412,
  'Text': "Software development has changed significantly over the years. Nowadays, it's common to use high-level programming languages like JavaScript, which are more developer-friendly. A simple JavaScript code to display a message in the console would be:\n\nconsole.log('Hello, World!');\n\nDespite their simplicity, high-level languages are powerful tools for creating complex applications.",
  'ContainsCode

## upon analysis we can see the problem is with the model learning wrong things from bad data to write the language of code

In [151]:
ef[ef["ID"].isin(ids)]

,ID,Text,ContainsCode,CodeList,gen_len,val
152,153,"In software development, it's not just about w...",,use strict',2,-9
411,412,Software development has changed significantly...,,"console.log('Hello, World!');",2,-10
729,730,The importance of commenting in code cannot be...,,,0,1
737,738,"In software engineering, a design pattern is a...",,,0,10
795,796,Software development methodologies have evolve...,,"struct { Start time.Time, End time.Time }",7,-19
1475,1476,Software development is a fascinating field. I...,,SystemoutprintHello World,2,-17
1638,1639,Software development is a dynamic field that i...,,\n\n\n#include<iostream>\n\nint main() {\n ...,7,-28
1645,1646,Software development is a complex and multifac...,,"function add(a: number, b: number): number {\n...",12,-21
1869,1870,Software development is a fascinating field. F...,,"System.out.println to print 'Hello, World!'",5,-21
1873,1874,Software development has a multitude of strate...,,std::cout,1,-7
